In [1]:
import pandas as pd
import yaml
import argparse
import torch
from model import YNet

In [2]:
%load_ext autoreload
%autoreload 2

#### Some hyperparameters and settings

In [3]:
CONFIG_FILE_PATH = 'config/atc_5_30.yaml'  # yaml config file containing all the hyperparameters
EXPERIMENT_NAME = 'atc_5_30'  # arbitrary name for this experiment
DATASET_NAME = 'atc'

TRAIN_DATA_PATH = 'data/ATC/atc_train_5_30.pkl'
TRAIN_IMAGE_PATH = 'data/ATC/train'
VAL_DATA_PATH = 'data/ATC/atc_test_5_30.pkl'
VAL_IMAGE_PATH = 'data/ATC/test'
OBS_LEN = 5  # in timesteps
PRED_LEN = 30  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

BATCH_SIZE = 4

#### Load config file and print hyperparameters

In [4]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'resize': 0.055,
 'batch_size': 8,
 'viz_epoch': 10,
 'learning_rate': 0.0001,
 'num_epochs': 100,
 'encoder_channels': [32, 32, 64, 64, 64],
 'decoder_channels': [64, 64, 64, 32, 32],
 'waypoints': [14, 29],
 'temperature': 1.8,
 'segmentation_model_fp': 'segmentation_models/SDD_segmentation.pth',
 'semantic_classes': 2,
 'loss_scale': 1000,
 'kernlen': 31,
 'nsig': 4,
 'use_features_only': False,
 'unfreeze': 100,
 'use_TTST': True,
 'rel_threshold': 0.002,
 'use_CWS': True,
 'CWS_params': {'sigma_factor': 6, 'ratio': 2, 'rot': True}}

#### Load preprocessed Data

In [5]:
df_train = pd.read_pickle(TRAIN_DATA_PATH)
df_val = pd.read_pickle(VAL_DATA_PATH)

In [6]:
df_train.head()

,trackId,frame,x,y,sceneId,metaId
0,15163400,450915,8247.2,5118.0,scene,0
1,15163400,450930,8264.7,5114.0,scene,0
2,15163400,450945,8253.1,5115.8,scene,0
3,15163400,450960,8264.2,5129.0,scene,0
4,15163400,450975,8247.3,5109.3,scene,0


#### Initiate model and load pretrained weights

In [7]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

/usr/local/anaconda3/envs/ynet-env/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'segmentation_models_pytorch.unet.model.Unet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/anaconda3/envs/ynet-env/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/anaconda3/envs/ynet-env/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' ha

#### Start training
Note, the Val ADE and FDE are without TTST and CWS to save time. Therefore, the numbers will be worse than the final values.

In [9]:
model.train(df_train, df_val, params, train_image_path=TRAIN_IMAGE_PATH, val_image_path=VAL_IMAGE_PATH,
            experiment_name=EXPERIMENT_NAME, batch_size=BATCH_SIZE, num_goals=NUM_GOALS, num_traj=NUM_TRAJ, 
            device=None, dataset_name=DATASET_NAME)

Preprocess data


Prepare Dataset: 100%|██████████| 1/1 [00:00<00:00, 153.71it/s]


Start training


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]


AttributeError: 'numpy.ndarray' object has no attribute 'to'